In [1]:
# https://runawayhorse001.github.io/LearningApacheSpark/clustering.html
from utilsom import * 

In [2]:
#input_data = spark.read.csv("ALOI_norm.csv", header=True, inferSchema=True, mode="DROPMALFORMED", encoding='UTF-8')
#df = df.drop("_c0")
#input_data.show(n=5,truncate=False)
#http://archive.ics.uci.edu/ml/datasets/HEPMASS  
input_data = load_data("hdfs://localhost:9000/data/ALOI_norm.csv")

In [29]:
print(input_data.count())
print(input_data)

50000
DataFrame[_c0: int, att1: double, att2: double, att3: double, att4: double, att5: double, att6: double, att7: double, att8: double, att9: double, att10: double, att11: double, att12: double, att13: double, att14: double, att15: double, att16: double, att17: double, att18: double, att19: double, att20: double, att21: double, att22: double, att23: double, att24: double, att25: double, att26: double, att27: double, id: double, c90: string]


In [3]:
# dellet label

df_input_data = input_data.drop('id','_c0','c90')

# staep 2
df_input_data = shuffling(df_input_data, 10)

# staep 3
input_features = assembler(df_input_data,df_input_data.columns[0:])

# staep 4
scaledData = scaler(input_features)

# SOM

In [37]:
#https://machinelearningnepal.com/2018/01/22/apache-spark-implementation-of-som-batch-algorithm/
# step 5 Som
from som.batch_som import SOM
from matplotlib import pyplot as plt
from matplotlib import patches as patches

#mapsize = calculate_map_size(input_data)
#print('Som: X = {} and Y = {}, {} neurones'.format(mapsize[0], mapsize[1],mapsize[0]*mapsize[1]))

#x =  mapsize[0] 
#y = mapsize[1]
x =  13
y = 6
som = SOM(x,y,27) # (x×y) neurons, (70 features)
som.train(scaledData, 10) # data , epochs  +10

   # for show All codebook
#som.net
   # dimention of codebook (x,y,features)
print('dimention of codebook 3D:',som.net.shape)

#som.net.reshpae(x*y,features)
data = som.net
data = data.transpose(2,1,0).reshape(-1,data.shape[2])
print('dimention of codebook 2D:',data.shape)

SOM training epoches 1
neighborhood radius  5.015752812467621
-------------------------------------
SOM training epoches 2
neighborhood radius  4.192962712629475
-------------------------------------
SOM training epoches 3
neighborhood radius  3.505144086407193
-------------------------------------
SOM training epoches 4
neighborhood radius  2.9301560515835217
-------------------------------------
SOM training epoches 5
neighborhood radius  2.449489742783178
-------------------------------------
SOM training epoches 6
neighborhood radius  2.0476725110792193
-------------------------------------
SOM training epoches 7
neighborhood radius  1.7117698594097055
-------------------------------------
SOM training epoches 8
neighborhood radius  1.4309690811052556
-------------------------------------
SOM training epoches 9
neighborhood radius  1.1962311988513155
-------------------------------------
SOM training epoches 10
neighborhood radius  1.0
-------------------------------------
dimentio

In [38]:
# prediction som
# pour avoir bmu, bmu, index 
pre = som.predict(scaledData)
pre.show(2)
print(pre.count())

+--------------+----------------+----------------+-----+----------------+----------------+-----+----------------+----------------+----------------+---------------+----------------+----------------+-----+----------------+----------------+---------------+----------------+----------------+----------------+----+----------------+----------------+----+----+----+----+--------------------+----------+--------------------+
|          att1|           att10|           att11|att12|           att13|           att14|att15|           att16|           att17|           att18|          att19|            att2|           att20|att21|           att22|           att23|          att24|           att25|           att26|           att27|att3|            att4|            att5|att6|att7|att8|att9|                 bmu|   bmu_idx|            features|
+--------------+----------------+----------------+-----+----------------+----------------+-----+----------------+----------------+----------------+---------------+---

In [39]:
# to kmenas, select, bmu 
bmux = pre.select('bmu','bmu_idx','features')
print(bmux.show(5))

+--------------------+----------+--------------------+
|                 bmu|   bmu_idx|            features|
+--------------------+----------+--------------------+
|[0.97341202528250...|[12.0,1.0]|[0.970369747642,0...|
|[0.59670496187115...| [6.0,5.0]|(27,[0,1,3,4,9,12...|
|[0.92026447592050...|[12.0,2.0]|[0.935771823365,3...|
|[0.97341202528250...|[12.0,1.0]|[0.981757180197,2...|
|[0.64172085599527...| [0.0,1.0]|[0.423589563303,0...|
+--------------------+----------+--------------------+
only showing top 5 rows

None


# K-means

In [40]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="bmu", \
                               outputCol="indexedbmu",\
                               ).fit(bmux)

data = featureIndexer.transform(bmux)

In [ ]:
data.show(5,True)

+--------------------+----------+--------------------+--------------------+
|                 bmu|   bmu_idx|            features|          indexedbmu|
+--------------------+----------+--------------------+--------------------+
|[0.97341202528250...|[12.0,1.0]|[0.970369747642,0...|[0.97341202528250...|
|[0.59670496187115...| [6.0,5.0]|(27,[0,1,3,4,9,12...|[0.59670496187115...|
|[0.92026447592050...|[12.0,2.0]|[0.935771823365,3...|[0.92026447592050...|
|[0.97341202528250...|[12.0,1.0]|[0.981757180197,2...|[0.97341202528250...|
|[0.64172085599527...| [0.0,1.0]|[0.423589563303,0...|[0.64172085599527...|
+--------------------+----------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans() \
          .setK(2) \
          .setFeaturesCol("indexedbmu")\
          .setPredictionCol("cluster")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, kmeans])

model = pipeline.fit(bmux)

cluster = model.transform(bmux)

In [ ]:
cluster = model.transform(bmux)

In [ ]:
cluster.show(5)

+--------------------+----------+--------------------+--------------------+-------+
|                 bmu|   bmu_idx|            features|          indexedbmu|cluster|
+--------------------+----------+--------------------+--------------------+-------+
|[0.97341202528250...|[12.0,1.0]|[0.970369747642,0...|[0.97341202528250...|      0|
|[0.59670496187115...| [6.0,5.0]|(27,[0,1,3,4,9,12...|[0.59670496187115...|      1|
|[0.92026447592050...|[12.0,2.0]|[0.935771823365,3...|[0.92026447592050...|      0|
|[0.97341202528250...|[12.0,1.0]|[0.981757180197,2...|[0.97341202528250...|      0|
|[0.64172085599527...| [0.0,1.0]|[0.423589563303,0...|[0.64172085599527...|      1|
+--------------------+----------+--------------------+--------------------+-------+
only showing top 5 rows



In [ ]:
y_pred = cluster.select('cluster')

In [ ]:
import numpy as np
y_pred = np.array(list(cluster.select('cluster').toPandas()['cluster']))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# load data
df = pd.read_csv('ALOI_norm.csv')  
df['c90'] = df.c90.apply(lambda label: 1 if label == "'yes'" else 0)
df.head()
del df['id']
del df['Unnamed: 0']
# df to values
df = df.values
# 
y_test= df[:,27]

In [ ]:
# predict
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of SOM K-means = {:.6f} %".format(acc * 100))
# Matrix de confusion
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

In [ ]:
Test Accuracy of SOM K-means = 59.864000 %
[[29272 19220]
 [  848   660]]


              precision    recall  f1-score   support

         0.0       0.97      0.60      0.74     48492
         1.0       0.03      0.44      0.06      1508

   micro avg       0.60      0.60      0.60     50000
   macro avg       0.50      0.52      0.40     50000
weighted avg       0.94      0.60      0.72     50000